<a href="https://colab.research.google.com/github/SaiMeghanath/AI_Career_Guidance_System/blob/main/AI_Powered_Career_Guidance_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**"AI-Powered Career Guidance System using Python Full Stack Technologies"**

---



**📌 1. Project Title and Introduction**

# 🎯 AI-Powered Career Guidance System
This project provides intelligent career recommendations based on user skills and interests using Python Full Stack technologies.  
It uses a rule-based AI engine built with Python and Flask, deployed publicly via ngrok.

---



# **2. Install Required Libraries**

** Step 1: Project Setup + Sample Career Dataset**




In [8]:
#  Install libraries (only if not already installed)
!pip install flask pyngrok --quiet






---



**3. Import Libraries and Prepare Dataset**

In [9]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd

# Sample dataset of careers
data = [
    {"career": "Data Scientist", "skills": ["Python", "Statistics", "Machine Learning"], "interests": ["Data Analysis", "AI", "Problem Solving"]},
    {"career": "Web Developer", "skills": ["HTML", "CSS", "JavaScript", "Python"], "interests": ["Design", "UI/UX", "Building Websites"]},
    {"career": "AI Researcher", "skills": ["Deep Learning", "Python", "NLP"], "interests": ["AI", "Research", "Mathematics"]},
    {"career": "Cloud Engineer", "skills": ["AWS", "Python", "Linux"], "interests": ["Cloud", "DevOps", "Infrastructure"]},
    {"career": "Business Analyst", "skills": ["Excel", "SQL", "Python"], "interests": ["Business", "Data", "Decision Making"]}
]

df = pd.DataFrame(data)
df




,career,skills,interests
0,Data Scientist,"[Python, Statistics, Machine Learning]","[Data Analysis, AI, Problem Solving]"
1,Web Developer,"[HTML, CSS, JavaScript, Python]","[Design, UI/UX, Building Websites]"
2,AI Researcher,"[Deep Learning, Python, NLP]","[AI, Research, Mathematics]"
3,Cloud Engineer,"[AWS, Python, Linux]","[Cloud, DevOps, Infrastructure]"
4,Business Analyst,"[Excel, SQL, Python]","[Business, Data, Decision Making]"




---



**4. Take User Input**

## Step 2: Take User Input for Skills and Interests


In [11]:
# Get user input (CLI-style)
user_skills = input("Enter your skills (comma-separated): ").strip().title().split(",")
user_interests = input("Enter your interests (comma-separated): ").strip().title().split(",")

user_skills = [skill.strip() for skill in user_skills]
user_interests = [interest.strip() for interest in user_interests]

print("✅ Your Skills:", user_skills)
print("✅ Your Interests:", user_interests)



Enter your skills (comma-separated): Pyhon,powerbi
Enter your interests (comma-separated): AI,ML
✅ Your Skills: ['Pyhon', 'Powerbi']
✅ Your Interests: ['Ai', 'Ml']


**5. Matching Logic: Score and Recommend Careers**

**## Step 3: Match Careers Based on User Input**


In [12]:
# Score careers
def score_career(row, user_skills, user_interests):
    skill_matches = len(set(row['skills']) & set(user_skills))
    interest_matches = len(set(row['interests']) & set(user_interests))
    return skill_matches + interest_matches

df['score'] = df.apply(lambda row: score_career(row, user_skills, user_interests), axis=1)
sorted_df = df.sort_values(by='score', ascending=False)

top_matches = sorted_df[sorted_df['score'] > 0].head(3)

if not top_matches.empty:
    print("\n🎯 Top Career Recommendations for You:\n")
    for index, row in top_matches.iterrows():
        print(f"👉 {row['career']} (Score: {row['score']})")
else:
    print("❌ No matching careers found.")



❌ No matching careers found.




---



**6. Flask API Setup**

**Step 4: Create Flask API Endpoint for Career Recommendations**


In [16]:
app = Flask(__name__)

@app.route("/recommend", methods=["POST"])
def recommend():
    data = request.get_json()
    user_skills = [s.strip().title() for s in data.get("skills", [])]
    user_interests = [i.strip().title() for i in data.get("interests", [])]

    def score_career(row):
        return len(set(row['skills']) & set(user_skills)) + len(set(row['interests']) & set(user_interests))

    df['score'] = df.apply(score_career, axis=1)
    top_matches = df[df['score'] > 0].sort_values(by='score', ascending=False).head(3)

    results = []
    for _, row in top_matches.iterrows():
        results.append({
            "career": row['career'],
            "score": row['score'],
            "matched_skills": list(set(row['skills']) & set(user_skills)),
            "matched_interests": list(set(row['interests']) & set(user_interests))
        })

    if results:
        return jsonify({"status": "success", "recommendations": results})
    else:
        return jsonify({"status": "no_match", "message": "No matching careers found."})




---



**7. Start Flask with Ngrok**

**Step 5: Run Flask App Publicly via Ngrok**


In [18]:
!ngrok config add-authtoken 2siOxN1Mf3nbgcYEJwF4YjEfbwM_4oDe2SNpHvt1JfcoDbv2j


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:
# 🔐 If not done already, run this once
# !ngrok config add-authtoken YOUR_AUTHTOKEN_HERE

public_url = ngrok.connect(5000)
print("🔥 Your Flask API is live at:", public_url)

app.run(port=5000)


🔥 Your Flask API is live at: NgrokTunnel: "https://26df-34-86-138-13.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


**8. Test the API with Python**

**Step 6: Test the API Using Python Requests**

In [ ]:
import requests

API_URL = "https://your-ngrok-url.ngrok-free.app/recommend"  # Replace with your URL

payload = {
    "skills": ["Python", "Flask", "HTML"],
    "interests": ["AI", "Web Development"]
}

response = requests.post(API_URL, json=payload)
print("Status code:", response.status_code)
print("Response JSON:", response.json())